<a href="https://colab.research.google.com/github/Sudipta-Mitra/PersuasionForGood/blob/main/PersuasionForGood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Persuasion Dialogue System for Social Good

Objective

To build a dialogue system that can generate persuasive responses encouraging users to act for social good — e.g., charity donation, environment awareness — using the PersuasionForGood Corpus and a language model (GPT-Neo) fine-tuned with reinforcement learning (RLHF).

In [ ]:
import pandas as pd
df=pd.read_csv('/content/full_dialog.csv')

In [ ]:
df.head()
# B2: Dialogue ID
# B4: Role (0 means persuader, 1 means persuadee)
# Turn: Turn index
# Unit: Sentence in utterance

,Unnamed: 0,Unit,Turn,B4,B2
0,0,Good morning. How are you doing today?,0,0,20180904-045349_715_live
1,1,Hi. I am doing good. How about you?,0,1,20180904-045349_715_live
2,2,I'm doing pretty good for a Tuesday morning.,1,0,20180904-045349_715_live
3,3,"Haha. Same here, but it really feels like a Mo...",1,1,20180904-045349_715_live
4,4,Ugh yes it does!,2,0,20180904-045349_715_live


In [ ]:
df = df.rename(columns={
    'Unit': 'text',
    'Turn': 'turn_id',
    'B4': 'role',
    'B2': 'dialogue_id'
})

# Drop unnecessary column
df = df.drop(columns=['Unnamed: 0'])

# Map roles for readability
df['role'] = df['role'].map({0: 'persuader', 1: 'persuadee'})

print(df.head())


                                                text  turn_id       role  \
0             Good morning. How are you doing today?        0  persuader   
1                Hi. I am doing good. How about you?        0  persuadee   
2      I'm doing pretty good for a Tuesday morning.         1  persuader   
3  Haha. Same here, but it really feels like a Mo...        1  persuadee   
4                                   Ugh yes it does!        2  persuader   

                dialogue_id  
0  20180904-045349_715_live  
1  20180904-045349_715_live  
2  20180904-045349_715_live  
3  20180904-045349_715_live  
4  20180904-045349_715_live  


In [ ]:
# Create input-output pairs
pairs = []
for i in range(len(df)-1):
    if df.iloc[i]['role'] == 'persuadee' and df.iloc[i+1]['role'] == 'persuader':
        pairs.append({
            'input_text': f"User: {df.iloc[i]['text']}",
            'target_text': df.iloc[i+1]['text']
        })

dataset = pd.DataFrame(pairs)
dataset.to_csv("formatted_pairs.csv", index=False)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

hf_dataset = Dataset.from_pandas(dataset)

In [ ]:
def preprocess(examples):
    # Concatenate input and target texts for each example in the batch
    texts = [examples["input_text"][i] + " " + examples["target_text"][i] for i in range(len(examples["input_text"]))]
    return tokenizer(texts, truncation=True, padding="max_length", max_length=256)

tokenizer.pad_token = tokenizer.eos_token
tokenized = hf_dataset.map(preprocess, batched=True)
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

args = TrainingArguments(
    output_dir="./neo_persuasion",
    per_device_train_batch_size=2,
    num_train_epochs=10,
    learning_rate=5e-5,
    logging_dir="./logs",
    save_strategy="epoch"
)

Map:   0%|          | 0/10331 [00:00<?, ? examples/s]

In [ ]:

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()
model.save_pretrained("./neo_persuasion_finetuned")
tokenizer.save_pretrained("./neo_persuasion_finetuned")


/tmp/ipython-input-3710156766.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sudiptamitra945 (sudiptamitra945-nshm) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,3.078300
1000,2.956200
1500,2.974800
2000,2.877900
2500,2.896600
3000,2.831700
3500,2.854000
4000,2.778800
4500,2.780700
5000,2.789200


('./neo_persuasion_finetuned/tokenizer_config.json',
 './neo_persuasion_finetuned/special_tokens_map.json',
 './neo_persuasion_finetuned/vocab.json',
 './neo_persuasion_finetuned/merges.txt',
 './neo_persuasion_finetuned/added_tokens.json',
 './neo_persuasion_finetuned/tokenizer.json')

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="./neo_persuasion_finetuned")

prompt = "User: I’m not sure if small donations make any difference."
response = generator(prompt, max_length=80, do_sample=True, temperature=0.8)
print(response[0]['generated_text'])


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


User: I’m not sure if small donations make any difference. Small donations really aren't as important as large donations.  Small donations can help a lot of children in need.  Small donations will indeed help a lot of children in need.  In the first two months of 2018 alone, 1,000 children were reportedly killed or injured in intensifying violence.   So not much difference can be made between small donations at this point.  Small donations are a good way to think about donating to a cause.  I know sometimes we are short on money here and short on time, but think about those children who need every bit you can give.   Small donations will indeed help a lot of children in need.          Small donations will indeed help a lot of children in need.  I can attest to that.  Even a little bit can do a lot, especially in war zones.     I know it will help some.   Small donations will indeed help many.  In Syria, 1,000 children have grown up facing the daily threat of violence.     To provide re

In [ ]:
def persuasion_reward(response):
    # Reward if response includes empathy or encouragement
    reward = 0
    if any(word in response.lower() for word in ["understand", "help", "together", "hope", "good", "difference"]):
        reward += 1
    if "donate" in response.lower():
        reward += 1
    return reward


In [ ]:
from trl import PPOTrainer, PPOConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from types import SimpleNamespace

In [ ]:
config = PPOConfig(stop_token_id=tokenizer.eos_token_id)
model = AutoModelForCausalLM.from_pretrained("./neo_persuasion_finetuned")
tokenizer = AutoTokenizer.from_pretrained("./neo_persuasion_finetuned")
tokenizer.model_max_length = 512 # Explicitly set model_max_length

# Explicitly set is_decoder to True for the model config
model.config.is_decoder = True

In [ ]:
# Create a dummy generation_config for the tokenizer as a workaround
tokenizer.generation_config = SimpleNamespace(eos_token_id=tokenizer.eos_token_id)

In [ ]:
!pip install graphviz --quiet
from graphviz import Digraph

dot = Digraph(comment="Persuasion Model Full Pipeline", format='png')
dot.attr(rankdir='TB', size='8,12')

# Define nodes
dot.node('A', 'Start / Import Libraries')
dot.node('B', 'Load Dataset (full_dialog.csv)')
dot.node('C', 'Clean & Rename Columns\nDrop unused, Map Roles')
dot.node('D', 'Create Input–Output Pairs\n(Persuadee → Persuader)')
dot.node('E', 'Tokenize & Preprocess Data')
dot.node('F', 'Fine-Tune GPT-Neo Model\n(Trainer + TrainingArguments)')
dot.node('G', 'Save Fine-Tuned Model')
dot.node('H', 'Test Model Response\n(Persuasion Generation)')
dot.node('I', 'Define Reward Function\n(Keyword-based scoring)')
dot.node('J', 'Initialize PPO Setup\n(Reinforcement Learning Prep)')
dot.node('K', 'Deploy with Streamlit\n(Interactive Chat UI)')
dot.node('L', 'End / Ready to Use')

# Define edges (flow)
dot.edges(['AB', 'BC', 'CD', 'DE', 'EF', 'FG', 'GH', 'HI', 'IJ', 'JK', 'KL'])

# Render and view
dot.render('/content/persuasion_pipeline_full', view=True)


'/content/persuasion_pipeline_full.png'